In [ ]:
import pandas as pd
import random
import requests as r
import datetime as dt
import time as t
import matplotlib.pyplot as plt

In [14]:
# Load the data
call_data = pd.read_excel("data\\processed\\SPY_ATM_Calls_MarketData_2005_2025.xlsx")
put_data = pd.read_excel("data\\processed\\SPY_ATM_Puts_MarketData_2005_2025.xlsx")
stock_data = pd.read_excel("data\\processed\\SPY_StockPriceHistory_2005_2025.xlsx")

# Preprocess the data
call_data['date'] = pd.to_datetime(call_data['t_date'])
put_data['date'] = pd.to_datetime(put_data['t_date'])
stock_data['date'] = pd.to_datetime(stock_data['date'])

call_data = call_data[['date', 'option_symbol', 'price_strike', 'call_put','Ask', 'Bid', 'expiration_date']].rename(columns={'price_strike': 'call_price'})
put_data = put_data[['date', 'option_symbol', 'price_strike','call_put','Ask', 'Bid', 'expiration_date']].rename(columns={'price_strike': 'put_price'})
stock_data = stock_data[['date', 'symbol', 'open','close','volume']]

# Filter at-the-money options by matching dates
call_data = pd.merge(call_data, stock_data, on='date')
call_data['strike_diff'] = call_data['call_price'] - call_data['close']
call_data = call_data[call_data['strike_diff'] >= 0]
call_data = call_data.sort_values(by=['date', 'strike_diff']).drop_duplicates(subset=['date'], keep='first')
call_data = call_data[['date','option_symbol', 'call_price', 'call_put', 'Ask', 'Bid','expiration_date','open','close']]
call_data.rename(columns={'close': 'stock_close_price',  'open' : 'stock_open_price'}, inplace=True)

put_data = pd.merge(put_data, stock_data, on='date')
put_data['strike_diff'] = put_data['close'] - put_data['put_price']
put_data = put_data[(put_data['strike_diff'] >= 0) | ((put_data['date'] == pd.Timestamp('2022-01-24')) & (put_data['put_price'] == 440))] # 2022-01-24 did not have an ATM 
put_data = put_data.sort_values(by=['date', 'strike_diff']).drop_duplicates(subset=['date'], keep='first')
put_data = put_data[['date', 'option_symbol', 'put_price', 'call_put','Ask', 'Bid','expiration_date', 'open','close']]
put_data.rename(columns={'close': 'stock_close_price', 'open' : 'stock_open_price'}, inplace=True)

# Merge datasets
data = pd.concat([call_data, put_data], ignore_index=True)
data['strike_price'] =  data.apply(lambda row: row['call_price'] if pd.isna(row['put_price']) else row['put_price'], axis=1)
data[['date', 'expiration_date']] = data[['date', 'expiration_date']].apply(pd.to_datetime)

desired_order = ['date', 'option_symbol', 'call_put', 'strike_price', 'Ask', 'Bid', 'expiration_date', 'stock_open_price', 'stock_close_price']
data = data[desired_order]
data = data.sort_values(by='date').reset_index(drop=True)
unique_dates = data['date'].unique()
sell_dates = pd.Series(unique_dates).shift(-1)

# Map sell dates back to the original DataFrame
date_to_sell_date = dict(zip(unique_dates, sell_dates))
data['sell_date'] = data['date'].map(date_to_sell_date)

data.head(7)


,date,option_symbol,call_put,strike_price,Ask,Bid,expiration_date,stock_open_price,stock_close_price,sell_date
0,2005-01-24,FYNLD,C,120.0,12.70,12.50,2007-12-22,117.09,116.55,2006-01-24
1,2005-01-24,FYNXC,P,115.0,9.20,9.00,2007-12-22,117.09,116.55,2006-01-24
2,2006-01-24,CYYLZ,C,130.0,16.40,15.80,2008-12-20,126.63,126.55,2007-01-24
3,2006-01-24,CYYXU,P,125.0,9.70,9.40,2008-12-20,126.63,126.55,2007-01-24
4,2007-01-24,FYNLO,C,145.0,19.80,19.40,2009-12-19,142.97,143.95,2008-01-24
5,2007-01-24,FYNXJ,P,140.0,8.60,8.30,2009-12-19,142.97,143.95,2008-01-24
6,2008-01-24,CYYLE,C,135.0,22.65,21.85,2010-12-18,134.48,134.99,2009-01-23


In [15]:
all_calls_puts = pd.read_csv("all_calls_puts.csv")
all_calls_puts= pd.DataFrame(all_calls_puts)

all_calls_puts = all_calls_puts.drop(columns=['Unnamed: 0','style', 'price_open', 'price_high','price_low','price'])
all_calls_puts = all_calls_puts.drop_duplicates()

# Convert `t_date` and `expiration_date` to datetime in all_calls_puts
all_calls_puts[['t_date', 'expiration_date']] = all_calls_puts[['t_date', 'expiration_date']].apply(pd.to_datetime)

#all_calls_puts = all_calls_puts[all_calls_puts['t_date'].dt.month]

# Step 3: Perform the join on the specified columns
data = pd.merge(
    data,
    all_calls_puts,
    left_on=['sell_date', 'strike_price', 'call_put'],
    right_on=['t_date','price_strike','call_put'],
    how='left',
    suffixes=('_data', '_all_calls_puts')
)

data = data[~((data['sell_date'] == '2012-01-24') & (data['option_symbol_all_calls_puts'] == 'SPY   141220C00130000'))]

data = pd.merge(
    data,
    stock_data,
    left_on= ['sell_date'],
    right_on=['date'],
    how='left',
)

final_format = ['date_x', 'option_symbol_data', 'strike_price',  'call_put', 'option_symbol_all_calls_puts', 'Ask_data', 'Bid_data', 'expiration_date_data', 'stock_open_price', 'stock_close_price', 'sell_date', 'Ask_all_calls_puts', 'Bid_all_calls_puts', 'open','close','volume' ]
data = data[final_format]
data = data.rename(columns={
    'date_x': 'buy_date',
    'option_symbol_data': 'option_symbol',
    'option_symbol_all_calls_puts': 'option_symbol_2',
    'call_put': 'call_put',
    'strike_price': 'strike_price',
    'Ask_data': 'buy_date_ask',
    'Bid_data': 'buy_date_bid',
    'expiration_date_data': 'expiration_date',
    'stock_open_price': 'buy_date_open',
    'stock_close_price': 'buy_date_close',
    'sell_date': 'sell_date',
    'Ask_all_calls_puts': 'sell_date_ask',
    'Bid_all_calls_puts': 'sell_date_bid',
    'open': 'sell_date_open',
    'close': 'sell_date_close',
    'volume': 'sell_date_volume'
})

data.head(8)


,buy_date,option_symbol,strike_price,call_put,option_symbol_2,buy_date_ask,buy_date_bid,expiration_date,buy_date_open,buy_date_close,sell_date,sell_date_ask,sell_date_bid,sell_date_open,sell_date_close,sell_date_volume
0,2005-01-24,FYNLD,120.0,C,FYNLD,12.70,12.50,2007-12-22,117.09,116.55,2006-01-24,18.10,17.70,126.63,126.55,52854100.0
1,2005-01-24,FYNXC,115.0,P,FYNXC,9.20,9.00,2007-12-22,117.09,116.55,2006-01-24,4.80,4.60,126.63,126.55,52854100.0
2,2006-01-24,CYYLZ,130.0,C,CYYLZ,16.40,15.80,2008-12-20,126.63,126.55,2007-01-24,25.00,24.50,142.97,143.95,55744100.0
3,2006-01-24,CYYXU,125.0,P,CYYXU,9.70,9.40,2008-12-20,126.63,126.55,2007-01-24,3.30,3.10,142.97,143.95,55744100.0
4,2007-01-24,FYNLO,145.0,C,FYNLO,19.80,19.40,2009-12-19,142.97,143.95,2008-01-24,13.00,12.55,134.48,134.99,258380832.0
5,2007-01-24,FYNXJ,140.0,P,FYNXJ,8.60,8.30,2009-12-19,142.97,143.95,2008-01-24,18.40,17.90,134.48,134.99,258380832.0
6,2008-01-24,CYYLE,135.0,C,CYYLE,22.65,21.85,2010-12-18,134.48,134.99,2009-01-23,1.58,1.30,80.90,83.11,386612928.0
7,2008-01-24,CYYXZ,130.0,P,CYYXZ,17.50,16.90,2010-12-18,134.48,134.99,2009-01-23,51.55,49.05,80.90,83.11,386612928.0


In [16]:
class InvestmentBacktest:
    def __init__(self, initial_balance=15000, annual_contribution=7000, call_allocation=0.1, put_allocation=0.1):
        """
        Initialize the backtesting strategy with customizable parameters.
        :param initial_balance: Starting portfolio balance
        :param annual_contribution: Annual amount added to the portfolio
        :param call_allocation: Percentage allocated to call options
        :param put_allocation: Percentage allocated to put options
        """
        self.initial_balance = initial_balance
        self.annual_contribution = annual_contribution
        self.call_allocation = call_allocation
        self.put_allocation = put_allocation
        self.portfolio_balance = initial_balance
        self.spy_shares = 0
        self.results = []
        self.data = None  # Placeholder for the provided dataframe
    
    def set_data(self, data):
        """Set the dataframe directly instead of loading from a file."""
        self.data = data.copy()
        self.data.sort_values(by='buy_date', inplace=True)
    
    def get_initial_params(self):
        """Returns the initial parameters for reference."""
        return {
            'Initial Balance': self.initial_balance,
            'Annual Contribution': self.annual_contribution,
            'Call Allocation': self.call_allocation,
            'Put Allocation': self.put_allocation
        }


In [ ]:
def generate_results_table(self):
        """Generate a structured table of backtest results with each year as a column."""
        results_df = pd.DataFrame(self.results)
        results_table = results_df.pivot_table(index=['date'], values=['portfolio_balance', 'num_calls', 'num_puts', 'num_spy_shares', 'returns'], aggfunc='sum')
        return results_table.T

In [ ]:
def calculate_options_positions(self, current_balance, call_price, put_price, shares_held):
        """
        Calculate the number of call and put options to buy based on current balance and shares held.
        :param current_balance: Current portfolio balance
        :param call_price: Price of the call option (buy_date_ask)
        :param put_price: Price of the put option (buy_date_ask)
        :param shares_held: Number of SPY shares held
        :return: Number of call options, number of put options, and total cost
        """
        # Determine the max number of options allowed based on shares held
        options_limit = shares_held // 100 + 1 if shares_held > 0 else 1

        # Calculate allocation for calls and puts
        call_allocation_amount = current_balance * self.call_allocation
        put_allocation_amount = current_balance * self.put_allocation

        # Calculate number of options to purchase (rounded down)
        num_calls = min(options_limit, int(call_allocation_amount // (call_price * 100)))
        num_puts = min(options_limit, int(put_allocation_amount // (put_price * 100)))

        # Calculate total cost for options
        total_cost = (num_calls * call_price * 100) + (num_puts * put_price * 100)

        return num_calls, num_puts, total_cost


In [ ]:
def calculate_returns(self, num_calls, num_puts, call_sell_price, put_sell_price, spy_sell_price, spy_buy_price):
        """
        Calculate the profit or loss for calls, puts, and SPY shares.
        :param num_calls: Number of call options held
        :param num_puts: Number of put options held
        :param call_sell_price: Sell price for the call option (sell_date_bid)
        :param put_sell_price: Sell price for the put option (sell_date_bid)
        :param spy_sell_price: Sell price of SPY shares (sell_date_open)
        :param spy_buy_price: Buy price of SPY shares (buy_date_open)
        :return: Dictionary containing returns for calls, puts, and SPY shares
        """
        # Calculate returns for options
        call_return = (num_calls * call_sell_price * 100) - (num_calls * call_sell_price * 100)
        put_return = (num_puts * put_sell_price * 100) - (num_puts * put_sell_price * 100)

        # Calculate returns for SPY shares
        spy_return = self.spy_shares * (spy_sell_price - spy_buy_price)

        # Return a breakdown of profits/losses
        return {
            'call_return': call_return,
            'put_return': put_return,
            'spy_return': spy_return
        }


In [ ]:
def run_backtest(self):
        """Execute the backtest over the entire dataset."""
        for _, row in self.data.iterrows():
            
            current_balance = self.portfolio_balance + self.annual_contribution

            num_calls, num_puts, options_cost = self.calculate_options_positions(
                current_balance,
                row['buy_date_ask'] if row['call_put'] == 'C' else 0,
                row['buy_date_ask'] if row['call_put'] == 'P' else 0, 
                self.spy_shares)
            
            remaining_balance = current_balance - options_cost

            spy_price = row['buy_date_open']

            num_spy_shares = int(remaining_balance // spy_price)

            self.spy_shares += num_spy_shares

            trade_returns = self.calculate_returns(
                num_calls, num_puts, row['sell_date_bid'] if row['call_put'] == 'C' else 0,
                row['sell_date_bid'] if row['call_put'] == 'P' else 0,
                row['sell_date_open'], row['buy_date_open'])
            
            self.portfolio_balance += sum(trade_returns.values())

            self.results.append({
                'date': row['buy_date'],
                'portfolio_balance': self.portfolio_balance,
                'num_calls': num_calls,
                'num_puts': num_puts,
                'num_spy_shares': self.spy_shares,
                'returns': trade_returns
            })
